<a href="https://colab.research.google.com/github/Saputoa21/Applied_ML_Spoiler_Detection/blob/main/Applied_ML_Movie_Spoiler_Detection_Group_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Applied ML - Movie Spoiler Detection**

## Group 5: Arina Antonova, Anastasiya Saputo, Juliana Schöner, Milica Djordjevic

### Project Description
The project ...


### Workflow

1.   Loading the Dataset
2.   Analysing the Dataset
3.   Classical ML
  1.   Decision Tree    
  2.   Random Forest
  3.   Naive Bayes
4.   NLMs
  1.   Fine-tuning general pre-trained model
  2.   Fine-tuning fine-tuned model on spoiler detection
5. Comparison

## **1. Loading Datasets**

The zip files have been recompressed as they can be uploaded to Guthub reposirtory and then used here via direact upload function, i.e. not the orginial zip files from Kaggle were used.

In [ ]:
import pandas as pd
import zipfile
import os
from google.colab import files

In [ ]:
uploaded = files.upload() # Manually upload your ZIP file

Saving IMDB_movie_details.json.zip to IMDB_movie_details.json (1).zip


In [ ]:
zip_filename = list(uploaded.keys())[0]  # Get uploaded ZIP file name

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall("json_data")  # Extract files to "json_data" folder

In [ ]:
json_folder = "json_data"
json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

In [ ]:
file_path = '/content/json_data/IMDB_movie_details.json'
with open(file_path, 'r', encoding='utf-8') as f:
    for i in range(5):  # Print first 5 lines
        print(f.readline())

{"movie_id": "tt0105112", "plot_summary": "Former CIA analyst, Jack Ryan is in England with his family on vacation when he suddenly witnesses an explosion outside Buckingham Palace. It is revealed that some people are trying to abduct a member of the Royal Family but Jack intervenes, killing one of them and capturing the other, and stops the plan in its tracks. Afterwards, he learns that they're Irish revolutionaries and the two men are brothers. During his court hearing the one that's still alive vows to get back at Jack but is sentenced and that seems to be the end of it. However, whilst the man is being transported, he is broken out. Jack learns of this but doesn't think there's anything to worry about. But, when he is at the Naval Academy someone tries to kill him. He learns that they are also going after his family and so he rushes to find them, safe but having also been the victims of a failed assassination. That's when Jack decides to rejoin the CIA, and they try to find the man

In [ ]:
df_plot_summaries = pd.read_json(file_path, lines=True)

In [ ]:
df_plot_summaries.tail()

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."
1569,tt5013056,Evacuation of Allied soldiers from the British...,1h 46min,"[Action, Drama, History]",8.1,2017-07-21,The film alternates between three different pe...
1570,tt0104014/,"For a while now, beautiful 24-year-old Diana B...",1h 33min,"[Comedy, Drama]",5.3,1992-02-21,
1571,tt0114142/,"The marriage of David Burgess, a senior execut...",1h 32min,"[Drama, Thriller]",4.0,1999-01-29,


In [ ]:
df_plot_summaries['plot_summary'][0]

"Former CIA analyst, Jack Ryan is in England with his family on vacation when he suddenly witnesses an explosion outside Buckingham Palace. It is revealed that some people are trying to abduct a member of the Royal Family but Jack intervenes, killing one of them and capturing the other, and stops the plan in its tracks. Afterwards, he learns that they're Irish revolutionaries and the two men are brothers. During his court hearing the one that's still alive vows to get back at Jack but is sentenced and that seems to be the end of it. However, whilst the man is being transported, he is broken out. Jack learns of this but doesn't think there's anything to worry about. But, when he is at the Naval Academy someone tries to kill him. He learns that they are also going after his family and so he rushes to find them, safe but having also been the victims of a failed assassination. That's when Jack decides to rejoin the CIA, and they try to find the man before he makes another attempt.         

Dataset with spoilers

In [ ]:
uploaded = files.upload() # Manually upload your ZIP file

KeyboardInterrupt: 

## **2. Analysing Dataset**

### Percentage of spoilers per genre

### Most common word for spoilers

### Average length of a spolier

### Average number of names in spoilers

## **Classical Machine Learning Algorythms**

### NLTK Clasifier - code from Liad's exercise

In [ ]:
import nltk

In [ ]:
def gender_features(first_name: str) -> dict[str, float]:
  # Which features can we use to detect the a spoiler?
  # Craft at least one feature and return it as a feature dictionary
  first_letter_unicode = ord(first_name[0])
  last_letter_unicode = ord(first_name[-1])
  name_length = len(first_name)
  for female_ending in unique_female_endings:
    if first_name[-3:] == female_ending:
      ends_with_unique_female_endings = 1
      break
    else:
      ends_with_unique_female_endings = 0

  for male_ending in unique_male_endings:
    if first_name[-3:] == male_ending:
      ends_with_unique_male_endings = 1
      break
    else:
      ends_with_unique_male_endings = 0

#ends_with_unique_female_endings = 1 if any(first_name.endswith(ending) for ending in unique_female_endings) else 0 # for female names
#ends_with_unique_male_endings = 1 if any(first_name.endswith(ending) for ending in unique_male_endings) else 0 # for male names

  return {
      'first_letter_unicode': first_letter_unicode,
      'last_letter_unicode': last_letter_unicode,
      'name_length': name_length,
      'ends_with_unique_female_endings': ends_with_unique_female_endings,
      'ends_with_unique_male_endings': ends_with_unique_female_endings
      }

In [ ]:
# function check
print(gender_features('Abagael'))
print(gender_features('Aaron'))

In [ ]:
feature_sets = [(gender_features(first_name), gender) for (first_name, gender) in labeled_names]

# We split the dataset to create a previously unseen 'exam' for the model:
train_set, test_set = feature_sets[500:], feature_sets[:500]

# And then, define and train the NLTK classifier
name_classifier = nltk.DecisionTreeClassifier.train(train_set)

In [ ]:
for name in ["Bill", "Melissa", "Barack", "Michelle", "Joe", "Jill", "Donald", "Ivanka"]:
  predicted_gender = name_classifier.classify(gender_features(name))
  print(f"the gender prediction for {name} is {predicted_gender}")

In [ ]:
print(nltk.classify.accuracy(name_classifier, test_set))

In [ ]:
print(name_classifier.pretty_format())

### Decision Tree from Scikit-Learn - code from Liad's homework

In [ ]:
import numpy as np

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from matplotlib import pyplot as plt

In [ ]:
# Maps to convert labels into numerical ids:

label2idx = {'male':0, 'female': 1}
idx2label = {0: 'male', 1: 'female'}

In [ ]:
X = male_names + female_names
y = [label2idx['male']] * len(male_names) + [label2idx['female']] * len(female_names)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=1234, stratify=y)

In [ ]:
assert len(X_train) == len(y_train)
assert len(X_test) == len(y_test)

In [ ]:
# Initializing Decision Tree:
clf = tree.DecisionTreeClassifier(max_depth=2, random_state=1234)

In [ ]:
#And of course, we need our Feature Function again.
#However, unlike NLTK, which required a dictionary of features, when working with SKLearn, we need to use a list of numerical features (a vector).

#Here, we just return the last letter of the name as a feature. The python function ord returns the unicode numerical code of the character.

def gender_feature_function(first_name: str) -> list[float]:
  """
  Given a first name, return the features to represent it.

  Args:
    first_name (string): a string representing a first name.

  Returns:
    A list of numerical features
  """
  last_letter_code = ord(first_name[-1])
  return [last_letter_code]
assert gender_feature_function("Željko") == [111] #print(ord('o'))!

In [ ]:
#Now we can use our feature function and convert our data into numerical features:

X_train = list(map(gender_feature_function, X_train))
X_test = list(map(gender_feature_function, X_test))
X_train[:10]
#[[101], [104], [121], [102], [101], [114], [110], [101], [97], [101]]

In [ ]:
#Before using our training data vector for training the model, we often need to convert it from a python list, into a NumPy array. NumPy is an excellent python pacakge for handling mathematical operations with Python.

X_train = np.array(X_train)
X_test = np.array(X_test)
X_train.shape
(5958, 1)
X_train[:10]
array([[101],
       [104],
       [121],
       [102],
       [101],
       [114],
       [110],
       [101],
       [ 97],
       [101]])

In [ ]:
#Let's take a peek and ensure our data is in the way we want it to be:

# zip is a python method to combine (merge) two lists into one
# with it, we can see our features, and the labels, together as a lsit of tuples
print(list(zip(X_train[:5], y_train[:5])))
[(array([101]), 0), (array([104]), 1), (array([121]), 1), (array([102]), 0), (array([101]), 1)]

In [ ]:
# And finally, we are ready to train our model:
clf = clf.fit(X_train, y_train)

In [ ]:
#Visualizing our tree
#We have trained a model, and now we want to understand how this model operates: Which features got more importance, which less, and in which way?

tree.plot_tree(clf, feature_names = ["last_letter"])
plt.show()

In [ ]:
#Now that we have a model, we can evaluate how well we did.
#SKLearn has many different metrics to test our model. For classification tasks, the most common are:
#Precision
#Recall
#F-Score
#Area under curve (AUC)
#Mathew's Correlation Coefficience (MCC)
#But many more exists, and it depends on the task in hand, to decide which one would represent our data set.

#Note: Be careful and don't blindly trust the metrics. Always be judgemental and stay critical about your results and achievements. Here's an excellent video about the topic - and why we should suspect our resutls.

predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
#We can use graphviz to visualize the tree we trained, to understand which features are more important, and therefore have more weight.
#Graphviz creates an image of the tree, which looks something like this:
import graphviz

dot_data = tree.export_graphviz(clf,
                                out_file=None,
                                feature_names=['last_letter'],
                                class_names=["male", "female"],
                                filled=True,
                                rounded=True,
                               )
graph = graphviz.Source(dot_data)
graph.view()
'Source.gv.pdf'


In [ ]:
#SuperTree
#SuperTree is a newer interactive visualization package that helps us exploring decision trees It's an open source package, and you can find more about it here: https://github.com/mljar/supertree

%pip install supertree --quiet
from supertree import SuperTree
super_tree = SuperTree(clf,
                       X_test,
                       y_test,
                       feature_names=['last_letter'],
                       target_names=["male", "female"])


In [ ]:
# show tree in your notebook
super_tree.show_tree()

### Random Forest

### Naive Bayes - Liad's Homework

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline

%matplotlib inline

## References

*   Dataset:  https://www.kaggle.com/datasets/rmisra/imdb-spoiler-dataset?resource=download&select=IMDB_reviews.json
*   ...
